In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import files

# Upload your dataset zip file
uploaded = files.upload()

# Unzip the dataset
!unzip dataset.zip -d dataset


unzip:  cannot find or open dataset.zip, dataset.zip.zip or dataset.zip.ZIP.


In [ ]:
# Define image size
img_size = (48, 48)

# Load images and labels
def load_images(folder):
    images = []
    labels = []
    for emotion in os.listdir(folder):
        if emotion.startswith('.'):
            continue
        label = os.path.join(folder, emotion)
        for filename in os.listdir(label):
            img = load_img(os.path.join(label, filename), target_size=img_size, grayscale=True)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(emotion)
    return np.array(images), np.array(labels)

# Load and preprocess images
data_folder = 'dataset'
images, labels = load_images(data_folder)

# Normalize pixel values
images = images / 255.0

# Convert labels to categorical
label_dict = {'happy': 0, 'Sad': 1, 'Angry': 2}
labels = [label_dict[label] for label in labels]
labels = to_categorical(labels, num_classes=3)


NameError: ignored

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
8/8 [==============================] - 3s 144ms/step - loss: 1.1212 - accuracy: 0.3427 - val_loss: 1.0855 - val_accuracy: 0.4286
Epoch 2/10
8/8 [==============================] - 1s 107ms/step - loss: 1.0993 - accuracy: 0.3347 - val_loss: 1.0915 - val_accuracy: 0.4286
Epoch 3/10
8/8 [==============================] - 1s 107ms/step - loss: 1.0855 - accuracy: 0.3871 - val_loss: 1.0904 - val_accuracy: 0.3175
Epoch 4/10
8/8 [==============================] - 1s 106ms/step - loss: 1.0800 - accuracy: 0.4556 - val_loss: 1.0910 - val_accuracy: 0.3016
Epoch 5/10
8/8 [==============================] - 1s 106ms/step - loss: 1.0593 - accuracy: 0.4516 - val_loss: 1.0897 - val_accuracy: 0.3016
Epoch 6/10
8/8 [==============================] - 1s 106ms/step - loss: 1.0329 - accuracy: 0.4798 - val_loss: 1.0977 - val_accuracy: 0.3016
Epoch 7/10
8/8 [==============================] - 1s 108ms/step - loss: 1.0110 - accuracy: 0.4758 - val_loss: 1.1119 - val_accuracy: 0.3333
Epoch 8/10
8/8 [====

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


2/2 [==============================] - 0s 30ms/step - loss: 1.1392 - accuracy: 0.3651
Test Loss: 1.139210820198059, Test Accuracy: 0.3650793731212616


In [ ]:
# Save the model
model.save('facial_expression_model.h5')


In [ ]:
# Clone the YOLOv7 repository
!git clone https://github.com/WongKinYiu/yolov7.git

# Navigate to the YOLOv7 directory
%cd yolov7

# Download the pre-trained weights file
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov7-tiny.cfg

# Download the configuration file
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov7-tiny.weights


Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 26.72 MiB/s, done.
Resolving deltas: 100% (518/518), done.
/content/yolov7/yolov7
--2023-11-14 06:10:46--  https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov7-tiny.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7562 (7.4K) [text/plain]
Saving to: ‘yolov7-tiny.cfg’

yolov7-tiny.cfg     100%[===================>]   7.38K  --.-KB/s    in 0s      

2023-11-14 06:10:46 (81.0 MB/s) - ‘yolov7-tiny.cfg’ saved [7562/7562]

--2023-11-14 06:10:46--  https://github.com/AlexeyAB/da

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import dlib

# Load YOLOv7 weights and configuration
net = cv2.dnn.readNet('yolov7-tiny.weights', 'yolov7-tiny.cfg')
layer_names = net.getUnconnectedOutLayersNames()


# Load facial landmarks model
facial_landmarks_model = load_model('facial_expression_model.h5')

# Initialize face detector from Dlib
face_detector = dlib.get_frontal_face_detector()

# Access webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv7 Object Detection
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Facial Landmarks Detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_frame)

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_roi = frame[y:y + h, x:x + w]

        # Preprocess face for facial landmarks model
        face_landmarks_input = cv2.resize(face_roi, (224, 224))
        face_landmarks_input = np.expand_dims(face_landmarks_input, axis=0) / 255.0

        # Predict facial landmarks
        facial_landmarks = facial_landmarks_model.predict(face_landmarks_input)[0]

        # Draw facial landmarks on the frame
        for i in range(0, len(facial_landmarks), 2):
            x, y = int(facial_landmarks[i] * w), int(facial_landmarks[i + 1] * h)
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # Draw bounding box around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Real-time Expression Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


OSError: ignored